## Monthly extraction of eia.gov data into MySQL database

In [ ]:
# !pip install EIA_python
# !pip install requests
# !pip3 install mysql-connector-python

In [4]:
#EIA.gov API key = jZDezVZZoADh3qx2Kh1QkJ5b6SP6NCzVP6J7hgdl

In [10]:
import pandas as pd
import eia
import requests

In [11]:

# api-endpoint
URL = "https://api.eia.gov/v2/electricity/electric-power-operational-data/data/?frequency=monthly&data[0]=total-consumption-btu&facets[location][]=US&facets[sectorid][]=99&start=2001-01&end=2022-11&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=10000&api_key=jZDezVZZoADh3qx2Kh1QkJ5b6SP6NCzVP6J7hgdl"
  
r = requests.get(url = URL)
  
# extracting data in json format
data = r.json()

In [12]:
print(data['response']['data'])

[{'period': '2022-11', 'location': 'US', 'stateDescription': 'U.S. Total', 'sectorid': 99, 'sectorDescription': 'All Sectors', 'fueltypeid': 'COL', 'fuelTypeDescription': 'coal, excluding waste coal', 'total-consumption-btu': 610.96244, 'total-consumption-btu-units': 'million MMBtu'}, {'period': '2022-11', 'location': 'US', 'stateDescription': 'U.S. Total', 'sectorid': 99, 'sectorDescription': 'All Sectors', 'fueltypeid': 'WWW', 'fuelTypeDescription': 'wood and wood wastes', 'total-consumption-btu': 90.52876, 'total-consumption-btu-units': 'million MMBtu'}, {'period': '2022-11', 'location': 'US', 'stateDescription': 'U.S. Total', 'sectorid': 99, 'sectorDescription': 'All Sectors', 'fueltypeid': 'WOO', 'fuelTypeDescription': 'waste oil and other oils', 'total-consumption-btu': 0.83429, 'total-consumption-btu-units': 'million MMBtu'}, {'period': '2022-11', 'location': 'US', 'stateDescription': 'U.S. Total', 'sectorid': 99, 'sectorDescription': 'All Sectors', 'fueltypeid': 'WOC', 'fuelTyp

In [13]:
fdata = data['response']['data']

In [14]:
dataus = pd.DataFrame(fdata)
dataus.to_csv('Electricity.csv')
cols = ['period','total-consumption-btu']
# data = data[data['period','total-consumption-btu']]
dataus

,period,location,stateDescription,sectorid,sectorDescription,fueltypeid,fuelTypeDescription,total-consumption-btu,total-consumption-btu-units
0,2022-11,US,U.S. Total,99,All Sectors,COL,"coal, excluding waste coal",610.96244,million MMBtu
1,2022-11,US,U.S. Total,99,All Sectors,WWW,wood and wood wastes,90.52876,million MMBtu
2,2022-11,US,U.S. Total,99,All Sectors,WOO,waste oil and other oils,0.83429,million MMBtu
3,2022-11,US,U.S. Total,99,All Sectors,WOC,waste coal,6.60665,million MMBtu
4,2022-11,US,U.S. Total,99,All Sectors,WNT,onshore wind turbine,371.01852,million MMBtu
...,...,...,...,...,...,...,...,...,...
4995,2013-05,US,U.S. Total,99,All Sectors,DFO,distillate fuel oil,5.14143,million MMBtu
4996,2013-05,US,U.S. Total,99,All Sectors,COW,all coal products,1280.48674,million MMBtu
4997,2013-05,US,U.S. Total,99,All Sectors,COL,"coal, excluding waste coal",1269.40110,million MMBtu
4998,2013-05,US,U.S. Total,99,All Sectors,BIT,bituminous coal,604.40266,million MMBtu


## Pushing extracted data from API to MySQL database hosted in AWS

In [25]:
import mysql.connector as mysql

# enter your server IP address
HOST = "173.249.22.93" 
# database name
DATABASE = "giga_fp"
# user data
USER = "root"
# user password
PASSWORD = "Gig@Sql0802$Adm/f"
# connect to MySQL server
db_connection = mysql.connect(host=HOST, database=DATABASE, user=USER, password=PASSWORD)
print("Connected to:", db_connection.get_server_info())

Connected to: 5.7.36-0ubuntu0.18.04.1


In [27]:
#Pushing data to database
dataus.to_sql(name=DATABASE, con=db_connection, if_exists = 'replace', index=False)

In [8]:
#fetch data from sql server
import mysql.connector

mydb = mysql.connector.connect(
  host=HOST,
  user=USER,
  password=PASSWORD,
  database=DATABASE
)

mycursor = mydb.cursor()

mycursor.execute("SELECT * FROM Electricity_cons")

myresult = mycursor.fetchall()

for x in myresult:
  print(x)

(0, '2022-11', 'US', 'U.S. Total/', 96, 'All Commercial', 'ALL', 'all fuels', 17.30775, 'million MMBtu')
(1, '2022-11', 'US', 'U.S. Total', 2, 'IPP Non-CHP', 'ALL', 'all fuels', 1219.66004, 'million MMBtu')
(2, '2022-11', 'US', 'U.S. Total', 3, 'IPP CHP', 'ALL', 'all fuels', 103.86586, 'million MMBtu')
(3, '2022-11', 'US', 'U.S. Total', 7, 'Industrial CHP', 'ALL', 'all fuels', 234.30065, 'million MMBtu')
(4, '2022-11', 'US', 'U.S. Total', 98, 'Electric Power', 'ALL', 'all fuels', 2867.31368, 'million MMBtu')
(5, '2022-11', 'US', 'U.S. Total', 5, 'Commercial CHP', 'ALL', 'all fuels', 11.93718, 'million MMBtu')
(6, '2022-11', 'US', 'U.S. Total', 90, 'Electric Power Sector Non-CHP', 'ALL', 'all fuels', 2763.44782, 'million MMBtu')
(7, '2022-11', 'US', 'U.S. Total', 1, 'Electric Utility', 'ALL', 'all fuels', 1543.78778, 'million MMBtu')
(8, '2022-11', 'US', 'U.S. Total', 4, 'Commercial Non-CHP', 'ALL', 'all fuels', 5.37057, 'million MMBtu')
(9, '2022-11', 'US', 'U.S. Total', 94, 'Independe